In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#load data
df=pd.read_csv("../input/netflix-shows/netflix_titles.csv")
df.head()

In [ ]:
df.info()



The dataframe has 7787 rows and 12 columns. We observe that there are missing values in some columns. Let's explore this in detail.

In [ ]:
df.isna().sum()

In [ ]:
#Calculating Null Rate for better understanding and decision making

for i in df.columns:
    null_rate = (df[i].isna().sum()/len(df))*100
    if null_rate > 0:
        print("{}'s null rate is {}%".format(i, round(null_rate,2)))
    


'Director' and 'Cast' columns have too many missing values, hence can be removed. 'Country' may prove to be a useful factor. So let's explore how we can fix that column. 'Date added' and 'rating' have very less number of NAs.

In [ ]:
#Handling NAs 
# We fill up NAs in the 'Country' column with the most frequently occuring Country in the data

df['country'].value_counts().to_frame()

United States is the most frequent country. We replace NAs with this.

In [ ]:
df['country'] = df['country'].fillna(df['country'].mode()[0])

df['country'].isna().sum()

We follow the same approach for 'date_added' and 'ratings' columns as well.

In [ ]:
df['date_added'].value_counts().to_frame()

In [ ]:
df['date_added']= df['date_added'].fillna(df['date_added'].mode()[0])
df['date_added'].isna().sum()

In [ ]:
df['rating'].value_counts().to_frame()

In [ ]:
df['rating'] = df['rating'].fillna(df['rating'].mode()[0])
df['rating'].isna().sum()

Let's drop 'Director' & 'Cast' columns

In [ ]:
df = df.drop(['director', 'cast'], axis=1)
df.info()

Let's create new columns 'year_added' and 'month-added' from the 'date_added' column

In [ ]:
df['date_added'] = pd.to_datetime(df['date_added'])
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

df['year_added'].astype('int')
df['month_added'].astype('int')

df.head()

**Data Visualization**

Movies vs TV shows

In [ ]:
plt.figure(figsize= (8,6))

ax = sns.countplot(x= 'type', data = df, order = df['type'].value_counts().index)
plt.title("Comparision of # TV Shows and Movies")

for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()



We see that the number of movies is far higher than the number of TV shows.

In [ ]:
#Let's try to get a distribution of movies produced by country - Top 10

movies = df[df['type']== 'Movie']

plt.figure(figsize=(12,8))
ax2 = sns.countplot(x='country',data=movies, order = movies['country'].value_counts().head(10).index)
plt.title("Number of movies produced by each country")

for p in ax2.patches:
    ax2.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()



No surprises there ! US produces most of the movies on Netflix followed by India.

In [ ]:
#Movies by Genre

plt.figure(figsize=(16,8))
ax5 = sns.countplot(x='listed_in',data=movies, order = movies['listed_in'].value_counts().head(10).index)
plt.title("Number of movies by genre")
plt.xticks(rotation=45)

for p in ax5.patches:
    ax5.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()

'Documentaries', 'Stand-up Comedy' and 'Dramas' seem to be the popular genre in movies. 

In [ ]:
#Let's try to get a distribution of TV shows produced by country - Top 10

TV = df[df['type']== 'TV Show']

plt.figure(figsize=(14,8))
ax3 = sns.countplot(x='country',data=TV, order = TV['country'].value_counts().head(10).index)
plt.title("Number of TV shows produced by each country")

for p in ax3.patches:
    ax3.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()


US and UK are the top producers of TV shows that's on  Netflix. The accent in the British shows make them more attractive ain't it? 

In [ ]:
#TV Shows by Genre

plt.figure(figsize=(16,8))
ax6 = sns.countplot(x='listed_in',data=TV, order = TV['listed_in'].value_counts().head(10).index)
plt.title("Number of TV shows by genre")
plt.xticks(rotation=45)

for p in ax6.patches:
    ax6.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()

Interesting that 'Kid's TV' is the top genre in TV shows

Let's try to find how Netflix content is distributed in terms of Ratings.

In [ ]:
# Distribution by rating


plt.figure(figsize=(14,8))
ax4 = sns.countplot(x='rating',data=df, order = df['rating'].value_counts().index)
plt.title("Netflix content distribution by rating")

for p in ax4.patches:
    ax4.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()


Most Netflix content is rated suitable for Mature Audience (TV-MA), followed by content unsuitable for children 13 or younger (TV-14). 

In [ ]:
#Content distribution by 'Year added'


plt.figure(figsize=(16,8))
ax7 = sns.countplot(x='year_added',data=df, order = df['year_added'].value_counts().index)
plt.title("Netflix content distribution by year added")

for p in ax7.patches:
    ax7.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()

Most content seem to be added in 2019 followed closely by 2020 and 2018

Does Netflix add more content around the holiday season ? 

In [ ]:
#Content distribution by 'Month added'


plt.figure(figsize=(16,8))
ax8 = sns.countplot(x='month_added',data=df, order = df['month_added'].value_counts().index)
plt.title("Netflix content distribution by month added")

for p in ax8.patches:
    ax8.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')
     
plt.show()

As expected, most content is added in the month of December and then followed by October, which may be due to Halloween.

Finally let's look at the pattern of content adddition in Netflix.

In [ ]:
df1 = df['year_added'].value_counts().reset_index()
df1 = df1.rename(columns = {'index':'year_added', 'year_added':'count'})
df1 = df1.sort_values('year_added')

movies1 = movies['year_added'].value_counts().reset_index()
movies1 = movies1.rename(columns = {'index':'year_added', 'year_added':'count'})
movies1 = movies1.sort_values('year_added')

TV1 = TV['year_added'].value_counts().reset_index()
TV1 = TV1.rename(columns = {'index':'year_added', 'year_added':'count'})
TV1 = TV1.sort_values('year_added')

s1 = go.Scatter(x=df1['year_added'], y=df1["count"], name="Total Content", marker=dict(color="black"))
s2 = go.Scatter(x=movies1['year_added'], y=movies1["count"], name="Movies", marker=dict(color="blue"))
s3 = go.Scatter(x=TV1['year_added'], y=TV1["count"], name="TV shows", marker=dict(color="green"))

data = [s1, s2, s3]

layout = go.Layout(title="Content addition pattern over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()


We observe that Movies are added at a higher rate than TV shows.The content addition has picked up big time from 2015.

***Hope you liked it. If so, Please Upvote ! Cheers !! ***